# Parties

# Different attributes 

In [1]:
'at_fault': #OK
'case_id': #OK
'cellphone_use': # Problem: une chiée de '1', '2', '3' => deviner à quoi ça correspond ?? 
'financial_responsibility': #OK
'hazardous_materials': #OK,
'id': #OK
'movement_preceding_collision': #OK
'other_associate_factor_1': #OK,
'other_associate_factor_2': #OK,
'party_age': pd.Int16Dtype(), # Problem: interpretation age 999 => pregnant
'party_drug_physical': str, # Problem: 585062 rows with G corresponding to nothing => replace by None ? 
'party_number': #OK
'party_safety_equipment_1': #OK,
'party_safety_equipment_2': #OK,
'party_sex': #OK
'party_sobriety': #OK
'party_type': #OK
'school_bus_related': #OK
'statewide_vehicle_type': #OK
'vehicle_make': # Koi kil faut check ?
'vehicle_year':#OK,

SyntaxError: invalid syntax (<ipython-input-1-16baf1a0f8e9>, line 1)

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle
import json

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
dtype_parties={
    'at_fault': bool, # bool or floats
    'case_id': str, # had to use str insteand of uint becaue we want leading zeros #cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'cellphone_use': str, 
    'financial_responsibility': str,
    'hazardous_materials': str,
    'id': int,
    'movement_preceding_collision': str,
    'other_associate_factor_1': str,
    'other_associate_factor_2': str,
    'party_age': pd.Int16Dtype(),
    'party_drug_physical': str,
    'party_number': int,
    'party_safety_equipment_1': str,
    'party_safety_equipment_2': str,
    'party_sex': str,
    'party_sobriety': str,
    'party_type': str,
    'school_bus_related': str,
    'statewide_vehicle_type': str,
    'vehicle_make': str,
    'vehicle_year': pd.Int16Dtype(),
}#check what type to give for integers
#int does not allow nulls
#np.uint64 allows up to 64 bytes (necessary for case id)
#pd.Int16/32/64Dtype() allows nulls

In [4]:
parties_data = pd.read_csv("CSV-2018/parties2018.csv",dtype=dtype_parties,usecols=dtype_parties.keys()) 

In [10]:
set(parties_data["cellphone_use"])

{'1', '2', '3', 'B', 'C', 'D', nan}

# Data cleaning

In [11]:
clean_data = parties_data.copy()

Translation tables for data cleaning

In [12]:
translation_movement_preceding_collision ={
    'stopped': "A",
    'proceeding straight': "B",
    'ran off road': "C",
    'making right turn': "D",
    'making left turn': "E",
    'making u-turn': "F",
    'backing': "G",
    'slowing/stopping': "H",
    'passing other vehicle': "I",
    'changing lanes': "J",
    'parking maneuver': "K",
    'entering traffic': "L",
    'other unsafe turning': "M",
    'crossed into opposing lane': "N",
    'parked': "O",
    'merging': "P",
    'traveling wrong way': "Q",
    'other': "R",
}

translation_party_sex ={
    'female': "F",
    'male': "M",
}

translation_party_type ={
    'driver': 1,
    'pedestrian': 2,
    'parked vehicle': 3,
    'bicyclist': 4, 
    'other': 5,
}

translation_statewide_vehicle_type ={
    'passenger car': "A",
    'passenger car with trailer':"B",
    'motorcycle or scooter': "C",
    'pickup or panel truck': "D",
    'pickup or panel truck with trailer': "E",
    'truck or truck tractor': "F",
    'truck or truck tractor with trailer': "G",
    'schoolbus': "H",
    'other bus': "I",
    'emergency vehicle': "J",
    'highway construction equipment': "K",
    'bicycle': "L",
    'other vehicle':"M",
    'pedestrian': "N",
    'moped':"O", 
}

Clean columns that need to be cleaned

In [13]:
clean_data["movement_preceding_collision"] = parties_data["movement_preceding_collision"].map(lambda s: translation_movement_preceding_collision.get(s, None))
clean_data["party_sex"] = parties_data["party_sex"].map(lambda s: translation_party_sex.get(s, None))
clean_data["party_type"] = parties_data["party_type"].map(lambda s: translation_party_type.get(s, None)).astype(pd.Int16Dtype())
clean_data["statewide_vehicle_type"] = parties_data["statewide_vehicle_type"].map(lambda s: translation_statewide_vehicle_type.get(s, None))


# Problems with data

### Cellphone use

In [14]:
cellphone_use_one = parties_data[parties_data["cellphone_use"] == '1']
cellphone_use_two = parties_data[parties_data["cellphone_use"] == '2']
cellphone_use_three = parties_data[parties_data["cellphone_use"] == '3']

cellphone_use_b = parties_data[parties_data["cellphone_use"] == 'B']
cellphone_use_c = parties_data[parties_data["cellphone_use"] == 'C']
cellphone_use_d = parties_data[parties_data["cellphone_use"] == 'D']

print(f"Set of values for cellphone_use: ")
set(parties_data["cellphone_use"])


Set of values for cellphone_use: 


{'1', '2', '3', 'B', 'C', 'D', nan}

In [15]:
numbers = len(cellphone_use_one) + len(cellphone_use_two) + len(cellphone_use_three)
letters = len(cellphone_use_b) + len(cellphone_use_c) + len(cellphone_use_d)
print(f"Occurences of 1,2,3: {numbers}")
print(f"Occurences of B,C,D: {letters}")


Occurences of 1,2,3: 2636894
Occurences of B,C,D: 2108830


We realised that the values that are stored in the cellphone_use column {'1', '2', '3', 'B', 'C', 'D', nan} are different to the ones on the handout {'B', 'C', 'D', nan}. The values that are in the data but not in the handout {'1', '2', '3'} appear 2636894 times. We decided not to drop these values because they are a big chunk of the data (56%). 

We needed to find a plausible mapping between the numbers and the letters. We opted to do it by doing a frequency analysis. As you can see in the next cell, it is clear that 1 and B are those that appear the least, and 3 and D are those that appear most frequently. 

Therefore, we concluded that the correct mapping is: 1 -> B, 2 -> C, 3 -> D



In [16]:
print(f"1 : {len(cellphone_use_one)} in % : {len(cellphone_use_one)/numbers:.3f} \t\t B : {len(cellphone_use_b)} in % : {len(cellphone_use_b)/letters:.3f}")
print(f"2 : {len(cellphone_use_two)} in % : {len(cellphone_use_two)/numbers:.3f} \t\t C : {len(cellphone_use_c)} in % : {len(cellphone_use_c)/letters:.3f}")
print(f"3 : {len(cellphone_use_three)} in % : {len(cellphone_use_three)/numbers:.3f} \t D : {len(cellphone_use_d)} in % : {len(cellphone_use_d)/letters:.3f}")


1 : 24787 in % : 0.009 		 B : 38932 in % : 0.018
2 : 39114 in % : 0.015 		 C : 795475 in % : 0.377
3 : 2572993 in % : 0.976 	 D : 1274423 in % : 0.604


As we imported the data in the database we chose to replace the None value by "D" since D already means "No Cell Phone/Unknown" which is equals to "no value".

In [27]:
translation_cell_phone_use = {
    "1": "B",
    "2": "C",
    "3": "D",
    None: "D"
}

clean_data["cellphone_use"] = parties_data["cellphone_use"].map(lambda s: translation_cell_phone_use.get(s, s))

In [28]:
set(clean_data["cellphone_use"])

{'B', 'C', 'D', nan}

### vehicle_make

Since "vehicle_make" is an open field there is a lot of errors and inconsistency. We corrected the most obvious typos (see below) and make some brands consistent. We chose not to modifiy this field too much since we are not experts in vehicle_make and that's error prone to do modify it manually. For example we decided not to remove values with "OTHER - ..." since they add information compared to a "None".

In [ ]:
vehicle_make_corrections = {
    "AMERICAN MOTORS" : "AMERICAN MOTORS (AMC)",
    "DODG": "DODGE",
    "HOND" : "HONDA",
    "MERCEDES BENZ":"MERCEDES-BENZ",
    "MAZD": "MAZDA",
    "TOYTA": "TOYOTA",
    "MISCELLANEOUS": None,
    "NOT STATED": None
}

In [ ]:
clean_data["vehicle_make"] = parties_data["vehicle_make"].map(lambda m: vehicle_make_corrections.get(m, m))

### party_drug_physical

We noticed 585062 rows of "party_drug_physical" with value "G" which is not an valid key for "party_drug_physical". We decided to replace is by None since we had no way to guess what the correct value.

In [ ]:
clean_data["party_drug_physical"] = parties_data["party_drug_physical"].map(lambda d: None if "G" == d else d)

# Save clean data in a pickle

In [ ]:
save_obj(clean_data, "clean_parties")

# Create CSV files

## Parties CVS

In [ ]:
translate = {
    "case_id": "collision_case_id", 
    "financial_responsibility": "financial_responsibility_id", 
    "movement_preceding_collision": "movement_preceding_collision_id",
    "party_drug_physical": "party_drug_physical_id",
    "party_sobriety": "party_sobriety_id",
    "party_type": "party_type_id",
    "statewide_vehicle_type": "statewide_vehicle_type_id",
    "cellphone_use": "cellphone_use_id"
}
unwanted_columns = [
    "party_safety_equipment_1", 
    "party_safety_equipment_2", 
    "other_associate_factor_1", 
    "other_associate_factor_2", 
    "party_number"]

parties_cvs = clean_data.copy().rename(columns=translate).drop(unwanted_columns, axis="columns")


In [ ]:
parties_cvs.to_csv('out/parties.csv', index=False)

## safety_equipment relation CVS

In [ ]:
translate = {"id": "party_id", "party_safety_equipment_1": "party_safety_equipment_id", "party_safety_equipment_2": "party_safety_equipment_id"}
safety_equipment_1 = clean_data[~clean_data["party_safety_equipment_1"].isnull()][["id", "party_safety_equipment_1"]].copy().rename(columns=translate)
safety_equipment_2 = clean_data[~clean_data["party_safety_equipment_2"].isnull()][["id", "party_safety_equipment_2"]].copy().rename(columns=translate)
safety_equipment_csv = pd.concat([safety_equipment_1, safety_equipment_2], axis=0)
sort_safety_equipment_csv = safety_equipment_csv.sort_values("party_id")

In [ ]:
sort_safety_equipment_csv.to_csv('out/party_equiped_with_safety_equipment.csv', index=False)

## Other_factor relation CSV

In [ ]:
translate = {"id": "party_id", "other_associate_factor_1": "other_associated_factor_id", "other_associate_factor_2": "other_associated_factor_id"}
other_associated_factor_1 = clean_data[~clean_data["other_associate_factor_1"].isnull()][["id", "other_associate_factor_1"]].copy().rename(columns=translate)
other_associated_factor_2 = clean_data[~clean_data["other_associate_factor_2"].isnull()][["id", "other_associate_factor_2"]].copy().rename(columns=translate)
other_associated_factor_csv = pd.concat([other_associated_factor_1, other_associated_factor_2], axis=0)
sort_other_associated_factor_csv = other_associated_factor_csv.sort_values("party_id")

In [ ]:
sort_other_associated_factor_csv.to_csv('out/party_associated_with_safety_other_associated_factor.csv', index=False)